In [17]:
import pandas as pd

df = pd.read_csv('bert_pipeline.csv')
df


,total_layer,train_layer,patience,sensitivity,specificity,accuracy,precision,mcc,micro_auroc,train_bce,valid_bce,valid_mcc
0,33,0,5,0.4577,0.9918,0.9642,0.7533,0.5706,0.9295,0.0058,0.1530,0.5926
1,33,1,5,0.5263,0.9901,0.9660,0.7432,0.6088,0.9177,0.0056,0.1690,0.6395
2,36,0,5,0.5199,0.9904,0.9660,0.7477,0.6070,0.9352,0.0010,0.1791,0.5800
3,36,1,5,0.5710,0.9888,0.9672,0.7366,0.6320,0.9450,0.0542,0.0966,0.6381
4,33,1,5,0.4609,0.9945,0.9669,0.8210,0.6007,0.9199,0.0128,0.1847,0.6199
5,33,2,5,0.5694,0.9893,0.9675,0.7437,0.6345,0.9402,0.0062,0.1622,0.6426
6,36,1,5,0.4530,0.9936,0.9656,0.7933,0.5841,0.9358,0.0016,0.1823,0.6209
7,36,2,5,0.5917,0.9889,0.9684,0.7450,0.6479,0.9359,0.0023,0.1630,0.6400
8,33,1,5,0.4785,0.9946,0.9679,0.8287,0.6155,0.9211,0.0014,0.2052,0.6316
9,33,2,5,0.4944,0.9938,0.9679,0.8136,0.6197,0.9305,0.0324,0.1247,0.6494


In [18]:
df_stat = df.groupby(['total_layer', 'train_layer', 'patience']).agg({
    'mcc': ['size', 'mean', 'std'],
       'sensitivity': ['mean', 'std'],
    #    'specificity': ['mean', 'std'],
    #    'accuracy': ['mean', 'std'],
       'precision': ['mean', 'std'],
       'valid_mcc': ['mean', 'std'],
    }
).reset_index()
df_stat[df_stat['train_layer'] > 0]

total_layer train_layer patience  mcc                     sensitivity  \
                                   size      mean       std        mean   
1          33           1        5    5  0.611620  0.007478    0.495380   
2          33           2        5    4  0.627575  0.007072    0.517925   
3          33           3        5    3  0.638100  0.004687    0.552867   
5          36           1        5    4  0.610825  0.026528    0.498025   
6          36           2        5    4  0.644825  0.008862    0.555450   
7          36           3        5    1  0.637900       NaN    0.563000   

            precision           valid_mcc            
        std      mean       std      mean       std  
1  0.026300  0.794420  0.033772  0.631040  0.007154  
2  0.036142  0.798825  0.037564  0.642050  0.007414  
3  0.031674  0.773733  0.033661  0.654700  0.001808  
5  0.059296  0.790700  0.038119  0.625650  0.008486  
6  0.035659  0.785500  0.031780  0.646475  0.005310  
7       NaN  0.759100       NaN  0.652700       NaN